# General Hospital Data - Find number of hospitals per county
### Cleaning the data

In [98]:
# Dependencies and Setup
import pandas as pd
import pathlib
import csv
import scipy.stats as st
import numpy as np
import calendar
from time import strptime

In [99]:
# Create paths for csv
general_hospitals = "../raw_data/hospital_dataset/general_hospitals.csv"

In [100]:
# Read csv
general_hospitals_df = pd.read_csv(general_hospitals)
pd.DataFrame(general_hospitals_df)

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
0,-94.945477,29.747620,8497,76777520,HOUSTON METHODIST SAN JACINTO HOSPITAL ALEXAND...,1700 JAMES BOWIE DRIVE,BAYTOWN,TX,77520,NOT AVAILABLE,...,2017-12-18T00:00:00.000Z,http://www.houstonmethodist.org/locations/san-...,NOT AVAILABLE,NOT AVAILABLE,48.0,NON-PROFIT,-999.0,182.0,NOT AVAILABLE,Y
1,-82.881843,40.027143,8498,129043230,"WOODS AT PARKSIDE,THE",349 OLDE RIDENOUR ROAD,COLUMBUS,OH,43230,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.thewoodsatparkside.com/,1815,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,50.0,NOT AVAILABLE,NOT AVAILABLE
2,-84.168027,39.774242,8499,130045404,DAYTON CHILDREN'S HOSPITAL,ONE CHILDRENS PLAZA,DAYTON,OH,45404,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.childrensdayton.org/cms/home/index....,1411,NOT AVAILABLE,39.0,NON-PROFIT,-999.0,155.0,PEDIATRIC LEVEL II,Y
3,-80.632972,41.005169,8500,128844512,VIBRA HOSPITAL OF MAHONING VALLEY,8049 SOUTH AVENUE,BOARDMAN,OH,44512,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.mahoningvalleyhospital.com/,1428,MAHONING VALLEY HOSPITAL BOARDMAN CAMPUS,39.0,PROPRIETARY,-999.0,45.0,NOT AVAILABLE,NOT AVAILABLE
4,-84.199398,39.747740,8501,129845417,HAVEN BEHAVIORAL SENIOR CARE OF DAYTON,"ONE ELIZABETH PLACE,E3 SUITE A",DAYTON,OH,45417,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,https://dayton.havenbehavioral.com/,1506,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,32.0,NOT AVAILABLE,NOT AVAILABLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7565,-80.701221,40.372434,8492,128243953,LIFE LINE HOSPITAL,200 SCHOOL STREET,WINTERSVILLE,OH,43953,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.llhospital.com/,1493,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,36.0,NOT AVAILABLE,NOT AVAILABLE
7566,-81.336345,40.401257,8493,127744621,"TEN LAKES CENTER, LLC","819 NORTH FIRST STREET,3RD FLOOR",DENNISON,OH,44621,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.tenlakescenter.com/,1469,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,16.0,NOT AVAILABLE,NOT AVAILABLE
7567,-84.294586,39.331523,8494,128345040,LINDNER CENTER OF HOPE,4075 OLD WESTERN ROW ROAD,MASON,OH,45040,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://lindnercenterofhope.org/,1481,NOT AVAILABLE,39.0,NON-PROFIT,-999.0,34.0,NOT AVAILABLE,NOT AVAILABLE
7568,-97.283341,29.112615,8495,13879022,CUERO COMMUNITY HOSPITAL,2550 NORTH ESPLANADE STREET,CUERO,TX,79022,NOT AVAILABLE,...,2017-12-18T00:00:00.000Z,http://www.cuerohosp.org,NOT AVAILABLE,NOT AVAILABLE,48.0,GOVERNMENT - DISTRICT/AUTHORITY,-999.0,49.0,LEVEL IV,Y


In [101]:
# remove all states that are not CA (California)
only_CA_df = general_hospitals_df.loc[(general_hospitals_df['STATE'] == 'CA') & (general_hospitals_df['COUNTRY'] == 'USA') & (general_hospitals_df['STATUS'] == 'OPEN')]
only_CA_df

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
593,-117.351005,34.073990,9093,2092324,ARROWHEAD REGIONAL MEDICAL CENTER,400 N. PEPPER AVENUE,COLTON,CA,92324,NOT AVAILABLE,...,2017-12-05T00:00:00.000Z,http://www.arrowheadmedcenter.org,NOT AVAILABLE,NOT AVAILABLE,6.0,GOVERNMENT - LOCAL,-999.0,456.0,II,Y
594,-117.325460,34.129372,9094,3092411,BALLARD REHABILITATION HOSP,1760 WEST 16TH STREET,SAN BERNARDINO,CA,92411,NOT AVAILABLE,...,2017-12-05T00:00:00.000Z,http://www.ballardrehab.com,NOT AVAILABLE,NOT AVAILABLE,6.0,PROPRIETARY,-999.0,60.0,NOT AVAILABLE,NOT AVAILABLE
595,-116.888208,34.246401,9095,3592315,BEAR VALLEY COMMUNITY HOSPITAL,41870 GARSTIN DRIVE,BIG BEAR LAKE,CA,92315,NOT AVAILABLE,...,2017-12-07T00:00:00.000Z,http://www.bvchd.com,NOT AVAILABLE,NOT AVAILABLE,6.0,GOVERNMENT - DISTRICT/AUTHORITY,-999.0,30.0,NOT AVAILABLE,Y
596,-117.687994,34.009083,9096,5291710,CANYON RIDGE HOSPITAL,5353 G STREET,CHINO,CA,91710,NOT AVAILABLE,...,2017-12-05T00:00:00.000Z,http://www.canyonridgehospital.com,NOT AVAILABLE,NOT AVAILABLE,6.0,PROPRIETARY,-999.0,106.0,NOT AVAILABLE,NOT AVAILABLE
597,-116.275139,34.131900,9097,16692252,HI-DESERT MEDICAL CENTER,6601 WHITE FEATHER ROAD,JOSHUA TREE,CA,92252,NOT AVAILABLE,...,2017-12-05T00:00:00.000Z,http://www.hdmc.org,NOT AVAILABLE,NOT AVAILABLE,6.0,GOVERNMENT - DISTRICT/AUTHORITY,-999.0,179.0,NOT AVAILABLE,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6227,-122.223390,37.488848,14165,19494063,KAISER FND HOSP - REDWOOD CITY,1100 VETERANS BOULEVARD,REDWOOD CITY,CA,94063,NOT AVAILABLE,...,2017-12-04T00:00:00.000Z,http://www.kaiserpermanente.org,NOT AVAILABLE,NOT AVAILABLE,6.0,NON-PROFIT,-999.0,149.0,NOT AVAILABLE,NOT AVAILABLE
6228,-122.454030,37.774039,14166,167684149,KENTFIELD HOSPITAL SAN FRANCISCO,450 STANYAN ST.,SAN FRANCISCO,CA,94117,NOT AVAILABLE,...,2017-12-04T00:00:00.000Z,http://www.kentfieldsanfrancisco.com/,NOT AVAILABLE,NOT AVAILABLE,6.0,PROPRIETARY,-999.0,60.0,NOT AVAILABLE,NOT AVAILABLE
6229,-122.405762,37.756576,14167,39094110,PRISCILLA CHAN & MARK ZUCKERBERG SAN FRANCISCO...,1001 POTRERO AVENUE,SAN FRANCISCO,CA,94110,NOT AVAILABLE,...,2017-12-04T00:00:00.000Z,https://sfgh.ucsf.edu/,NOT AVAILABLE,SAN FRANCISCO GENERAL HOSPITAL,6.0,GOVERNMENT - LOCAL,-999.0,375.0,I,NOT AVAILABLE
6230,-121.774458,37.249021,14168,167684183,SAN JOSE BEHAVIORAL HEALTH,455 SILICON VALLEY BOULEVARD,SAN JOSE,CA,95138,NOT AVAILABLE,...,2017-12-04T00:00:00.000Z,https://www.sanjosebh.com,NOT AVAILABLE,NOT AVAILABLE,6.0,PROPRIETARY,-999.0,80.0,NOT AVAILABLE,NOT AVAILABLE


In [102]:
# List out all the columns names to determine what compenents are necessary 
only_CA_df.columns.tolist()

['X',
 'Y',
 'OBJECTID',
 'ID',
 'NAME',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'ZIP4',
 'TELEPHONE',
 'TYPE',
 'STATUS',
 'POPULATION',
 'COUNTY',
 'COUNTYFIPS',
 'COUNTRY',
 'LATITUDE',
 'LONGITUDE',
 'NAICS_CODE',
 'NAICS_DESC',
 'SOURCE',
 'SOURCEDATE',
 'VAL_METHOD',
 'VAL_DATE',
 'WEBSITE',
 'STATE_ID',
 'ALT_NAME',
 'ST_FIPS',
 'OWNER',
 'TTL_STAFF',
 'BEDS',
 'TRAUMA',
 'HELIPAD']

In [103]:
# Delete extraneous columns
delete_extraneous_columns = only_CA_df.drop(columns= [
    "X"
    , "Y"
    , "ID"
    , "NAME"
    , "ADDRESS"
    , "CITY"
    , "STATE"
    , "ZIP4"
    , "SOURCE"
    , "NAICS_CODE"
    , "VAL_METHOD"
    , "TELEPHONE"
    , "WEBSITE"
    , "STATE_ID"
    , "ALT_NAME"
    , "ST_FIPS"
    , "OWNER"
    , "TTL_STAFF" 
    , "BEDS"
    , "TRAUMA"
    , "HELIPAD"
    , "COUNTYFIPS"
    , "SOURCEDATE"
    , "VAL_DATE"
    , "COUNTRY"
    , "ZIP"
    , "OBJECTID"
    , "TYPE"
    , "STATUS"
    , "POPULATION"
    , "LATITUDE"
    , "LONGITUDE"
    , "NAICS_DESC"
])    
delete_extraneous_columns.head()

,COUNTY
593,SAN BERNARDINO
594,SAN BERNARDINO
595,SAN BERNARDINO
596,SAN BERNARDINO
597,SAN BERNARDINO


### New Dataframe - Number of Hospitals per Counties

In [104]:
# Create a new dataframe
new_general_hospital_df = delete_extraneous_columns
new_general_hospital_df

,COUNTY
593,SAN BERNARDINO
594,SAN BERNARDINO
595,SAN BERNARDINO
596,SAN BERNARDINO
597,SAN BERNARDINO
...,...
6227,SAN MATEO
6228,SAN FRANCISCO
6229,SAN FRANCISCO
6230,SANTA CLARA


In [105]:
# Rename 0 to number of hospitals by county
rename_new_df = new_general_hospital_df.rename(columns={"COUNTY":"countOfHospitals"})
rename_new_df

,countOfHospitals
593,SAN BERNARDINO
594,SAN BERNARDINO
595,SAN BERNARDINO
596,SAN BERNARDINO
597,SAN BERNARDINO
...,...
6227,SAN MATEO
6228,SAN FRANCISCO
6229,SAN FRANCISCO
6230,SANTA CLARA


In [106]:
rename_new_df.astype(str)

,countOfHospitals
593,SAN BERNARDINO
594,SAN BERNARDINO
595,SAN BERNARDINO
596,SAN BERNARDINO
597,SAN BERNARDINO
...,...
6227,SAN MATEO
6228,SAN FRANCISCO
6229,SAN FRANCISCO
6230,SANTA CLARA


In [107]:
# Lower case all the letters 
list_hospital_df = rename_new_df["countOfHospitals"].str.title()
pd.DataFrame(list_hospital_df)

,countOfHospitals
593,San Bernardino
594,San Bernardino
595,San Bernardino
596,San Bernardino
597,San Bernardino
...,...
6227,San Mateo
6228,San Francisco
6229,San Francisco
6230,Santa Clara


In [108]:
# Number of counties 
list_hospital_df.nunique()

57

In [109]:
# Number of hospital for each county
total_hospital_bycounty = list_hospital_df.value_counts()
pd.DataFrame(total_hospital_bycounty)

,countOfHospitals
Los Angeles,128
Orange,37
San Diego,36
San Bernardino,31
Alameda,26
Riverside,24
Sacramento,18
San Francisco,17
Santa Clara,17
Kern,15


In [110]:
# Rename 0 to number of hospitals by county
# total_hospital_bycounty.rename(columns={"COUNTY":"countyName"})

In [111]:
# Final cleaned csv to be used
total_hospital_bycounty.to_csv("../cleaned_data/by_county_count_hospital.csv")